In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,9,11,12,13,16,...,48,49,54,55,56,57,59,60,62,63
2,26.5,0.0,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,...,0.00,-0.00,0.0,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00
3,0.0,9.1,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,...,0.00,-0.00,-0.0,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
4,-0.0,-0.0,37.02,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,...,-0.00,-0.00,-0.0,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,-0.0,0.0,-0.00,28.37,0.00,-0.00,0.00,0.00,0.00,-0.00,...,-0.00,0.00,0.0,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00
6,0.0,-0.0,-0.00,0.00,15.52,-0.00,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,0.0,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
9,0.0,-0.0,0.00,-0.00,-0.00,4.27,-0.00,-0.00,0.00,0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
11,0.0,0.0,0.00,0.00,-0.00,-0.00,13.21,0.00,0.00,0.00,...,0.00,0.00,0.0,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00
12,0.0,-0.0,-0.00,0.00,0.00,-0.00,0.00,9.28,-0.00,-0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00
13,-0.0,-0.0,-0.00,0.00,0.00,0.00,0.00,-0.00,11.06,-0.00,...,0.00,-0.00,0.0,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00
16,-0.0,0.0,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,8.11,...,-0.00,0.00,-0.0,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,9,11,12,13,16,...,48,49,54,55,56,57,59,60,62,63
2,1.00000,0.00002,-0.00001,-0.00002,0.00002,0.00001,0.00004,0.00000,-0.00001,-0.00002,...,0.00001,-0.00002,0.00004,0.00000,-0.00002,-0.00004,0.00003,-0.00000,0.00004,-0.00002
3,0.00002,1.00000,-0.00001,0.00002,-0.00002,-0.00000,0.00004,-0.00001,-0.00000,0.00005,...,0.00001,-0.00004,-0.00001,-0.00002,0.00000,0.00002,-0.00004,0.00004,-0.00004,-0.00001
4,-0.00001,-0.00001,1.00000,-0.00005,-0.00001,0.00002,0.00003,-0.00010,-0.00001,0.00000,...,-0.00002,-0.00002,-0.00006,-0.00002,0.00004,0.00005,0.00001,0.00007,0.00003,0.00002
5,-0.00002,0.00002,-0.00005,1.00000,0.00001,-0.00004,0.00005,0.00006,0.00003,-0.00004,...,-0.00001,0.00000,0.00000,-0.00004,0.00001,0.00002,-0.00001,-0.00005,-0.00001,0.00004
6,0.00002,-0.00002,-0.00001,0.00001,1.00000,-0.00003,-0.00000,0.00000,0.00002,-0.00001,...,-0.00006,0.00003,0.00002,-0.00006,-0.00006,-0.00004,-0.00001,0.00003,0.00003,-0.00006
9,0.00001,-0.00000,0.00002,-0.00004,-0.00003,1.00000,-0.00002,-0.00004,0.00006,0.00001,...,-0.00001,-0.00001,0.00008,-0.00002,-0.00003,-0.00001,-0.00003,0.00003,0.00001,-0.00003
11,0.00004,0.00004,0.00003,0.00005,-0.00000,-0.00002,1.00000,0.00008,0.00002,0.00005,...,0.00000,0.00004,0.00000,-0.00001,-0.00000,-0.00003,0.00002,-0.00005,0.00004,0.00006
12,0.00000,-0.00001,-0.00010,0.00006,0.00000,-0.00004,0.00008,1.00000,-0.00004,-0.00004,...,-0.00002,-0.00002,0.00005,-0.00003,-0.00003,0.00001,0.00002,-0.00000,-0.00004,0.00003
13,-0.00001,-0.00000,-0.00001,0.00003,0.00002,0.00006,0.00002,-0.00004,1.00000,-0.00001,...,0.00002,-0.00002,0.00005,0.00001,-0.00002,0.00004,-0.00004,0.00002,-0.00002,-0.00001
16,-0.00002,0.00005,0.00000,-0.00004,-0.00001,0.00001,0.00005,-0.00004,-0.00001,1.00000,...,-0.00001,0.00003,-0.00003,0.00005,0.00002,-0.00001,0.00001,0.00003,-0.00002,0.00003


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.01599220862128838

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.32517271e+01 2.65138582e+01 2.57519490e+01 2.50024609e+01
 2.40069492e+01 2.37115607e+01 2.29997902e+01 2.13669050e+01
 1.92832484e+01 1.84678278e+01 1.81580774e+01 1.51254096e+01
 1.17133834e+01 7.42800549e+00 5.46400244e+00 4.79686769e+00
 3.82599822e+00 3.37265677e+00 3.10543692e+00 2.72831093e+00
 2.44986850e+00 2.08956100e+00 1.85906728e+00 1.57706089e+00
 1.40125065e+00 1.09456447e+00 8.14089257e-01 7.21716373e-01
 6.25803176e-01 5.01572805e-01 1.14625785e-01 6.88390837e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.033227,-0.005790,0.993002,-0.042548,-0.011283,-0.002573,-0.009043,-0.005977,-0.007266,-0.005086,...,-0.003979,-0.026935,-0.002319,-0.006697,-0.036669,-0.008384,-0.024271,-0.000734,-0.025108,-0.002902
1,-0.041631,-0.002325,0.012490,-0.313834,-0.004983,-0.000966,-0.003778,-0.002354,-0.002908,-0.001992,...,-0.001516,-0.021154,-0.000884,-0.002712,-0.066953,-0.003506,-0.016495,-0.000286,-0.018377,-0.001117
2,-0.056197,-0.002563,0.013160,0.858738,-0.005604,-0.001106,-0.004190,-0.002580,-0.003248,-0.002257,...,-0.001703,-0.026019,-0.000984,-0.003041,-0.107029,-0.003897,-0.020168,-0.000333,-0.021977,-0.001218
3,-0.111182,-0.003999,0.019141,0.289815,-0.008797,-0.001749,-0.006751,-0.004093,-0.005166,-0.003520,...,-0.002706,-0.044543,-0.001527,-0.004761,-0.294950,-0.006161,-0.033245,-0.000492,-0.037240,-0.001956
4,-0.050125,-0.001087,0.004492,0.036133,-0.002457,-0.000490,-0.001807,-0.001134,-0.001396,-0.000932,...,-0.000717,-0.014155,-0.000402,-0.001269,0.795661,-0.001682,-0.010837,-0.000139,-0.011305,-0.000493
5,0.102091,0.001695,-0.007088,-0.050552,0.003963,0.000735,0.002944,0.001728,0.002253,0.001490,...,0.001140,0.024254,0.000666,0.002003,-0.280440,0.002618,0.016735,0.000212,0.019572,0.000826
6,0.761823,0.004436,-0.017356,-0.094488,0.010167,0.001885,0.007602,0.004559,0.005741,0.003839,...,0.002980,0.071654,0.001689,0.005244,-0.244910,0.006850,0.048532,0.000532,0.055259,0.002147
7,0.570444,-0.013441,0.045033,0.171182,-0.031695,-0.005685,-0.023316,-0.013831,-0.017600,-0.011718,...,-0.008954,-0.386424,-0.005047,-0.015984,0.276475,-0.021247,-0.201689,-0.001617,-0.245025,-0.006471
8,0.084409,-0.005850,0.015968,0.045354,-0.014357,-0.002438,-0.010325,-0.006015,-0.007699,-0.005032,...,-0.003817,0.842426,-0.002160,-0.006958,0.060377,-0.009406,-0.214909,-0.000702,-0.400834,-0.002790
9,0.034804,-0.003183,0.007911,0.021036,-0.007957,-0.001310,-0.005665,-0.003311,-0.004197,-0.002774,...,-0.002095,0.120991,-0.001174,-0.003778,0.026659,-0.005133,-0.362482,-0.000379,0.774024,-0.001480


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([1.03322664, 1.00232492, 0.98684005, 0.71018472, 1.00245712,
       0.9992646 , 0.99239791, 1.01383062, 1.00769894, 1.00277431,
       0.99414186, 0.97117118, 0.90384306, 1.03116332, 1.01149783,
       1.02218176, 1.02923819, 0.99389514, 0.26988683, 0.99482013,
       0.98141938, 0.99561074, 1.17849957, 0.98763997, 0.97358707,
       0.90929141, 0.99165304, 1.00822629, 1.00981574, 0.9780813 ,
       1.04228974, 0.79872518])